# ***Woking Enviroment***

In [1]:
from google.colab import drive
drive.mount('/content/drive') # fix the typo in the path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks
!ls

/content/drive/MyDrive/Colab Notebooks
amazon_alexa.tsv  CL1_project.ipynb


# import DATASETS

In [3]:
import pandas as pd
data = pd.read_csv('amazon_alexa.tsv', sep='\t')
data.head(10)


,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
5,5,31-Jul-18,Heather Gray Fabric,I received the echo as a gift. I needed anothe...,1
6,3,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many ...",1
7,5,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'...,1
8,5,30-Jul-18,Heather Gray Fabric,looks great,1
9,5,30-Jul-18,Heather Gray Fabric,Love it! I’ve listened to songs I haven’t hear...,1


In [4]:
mydata = data[['verified_reviews','feedback']]
mydata.columns = ['review','label']

mydata.head()

,review,label
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [ ]:
# Count the occurrences of each label
label_counts = mydata["label"].value_counts()

# Get the number of rows to drop from the majority class
rows_to_drop = label_counts.max() - label_counts.min()

# Drop rows from the majority class randomly
if rows_to_drop > 0:
   data_majority = mydata[mydata["label"] == 1]
   data_balanced = mydata.drop(data_majority.sample(rows_to_drop).index)
else:
   data_balanced = mydata.copy()

# Check the new class balance
print(data_balanced["label"].value_counts())

label
1    257
0    257
Name: count, dtype: int64


## DATA Processing

In [5]:
import re

def clean_text(text):
  # Remove special characters and punctuation
  text = re.sub(r"[^\w\s]", " ", text)

  # Remove single characters
  text = re.sub(r"\b[a-zA-Z]\b", " ", text)

  # Remove HTML tags
  text = re.sub(r"<[^>]*>", " ", text)

  # Lowercase the text
  text = text.lower()

  # Remove extra whitespace
  text = re.sub(r"\s+", " ", text)

  # Trim leading and trailing spaces
  text = text.strip()

  return text

In [8]:
import pandas as pd

# Count the occurrences of each label
label_counts = mydata["label"].value_counts()

# Get the number of rows to drop from the majority class
rows_to_drop = label_counts.max() - label_counts.min()

# Drop rows from the majority class randomly
if rows_to_drop > 0:
   data_majority = mydata[mydata["label"] == 1]
   data_balanced = mydata.drop(data_majority.sample(rows_to_drop).index)
else:
   data_balanced = mydata.copy()

# Check the new class balance
print(data_balanced["label"].value_counts())

label
1    257
0    257
Name: count, dtype: int64


In [ ]:
data_balanced

,review,label,clean_reviews
3,I have had a lot of fun with this thing. My 4 ...,1,have had lot of fun with this thing my 4 yr ol...
5,I received the echo as a gift. I needed anothe...,1,received the echo as gift needed another bluet...
10,"I sent it to my 85 year old Dad, and he talks ...",1,sent it to my 85 year old dad and he talks to ...
31,I like it,1,like it
40,My husband likes being able to use it to liste...,1,my husband likes being able to use it to liste...
...,...,...,...
3081,My 3rd echo device. Very easy to set up. Works...,1,my 3rd echo device very easy to set up works p...
3091,I didn’t order it,0,didn order it
3094,Amazing product. Still learnimg all the things...,1,amazing product still learnimg all the things ...
3096,The product sounded the same as the emoji spea...,0,the product sounded the same as the emoji spea...


## Data Split

In [9]:
import pandas as pd

# Assuming your DataFrame is called "df"
total_rows = len(data_balanced)
test_size = int(total_rows * 0.95)

# Randomly sample train_size rows for the training set
test_set = data_balanced.sample(test_size)

# Get the remaining rows for the test set
train_set = data_balanced.drop(test_set.index)

## Sentiment w/ LLM

### Setting up Gemini API

In [ ]:
!pip install -q -U google-generativeai

In [10]:
# Necessary packages
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

In [11]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
Google_API_KEY=userdata.get('Google_API_KEY')

genai.configure(api_key=Google_API_KEY)

In [12]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-1.5-flash-002


In [13]:
model = genai.GenerativeModel('gemini-pro')

In [14]:
%%time
response = model.generate_content("What is the meaning of life?")

to_markdown(response.text)

CPU times: user 180 ms, sys: 12.6 ms, total: 193 ms
Wall time: 14.6 s


> There is no one definitive answer to the question of the meaning of life. Different people have different beliefs and perspectives on what gives life meaning and purpose. Some common themes that emerge in discussions about the meaning of life include:
> 
> * **Finding purpose and fulfillment in one's work, hobbies, and relationships.** Many people find meaning in life through their careers, creative pursuits, and connections with others.
> 
> * **Making a difference in the world.** Some people find meaning in life through volunteer work, activism, or other forms of service to their communities or to society as a whole.
> 
> * **Living in accordance with one's values.** For some people, living a meaningful life means living in a way that is consistent with their moral compass and personal beliefs.
> 
> * **Experiencing joy, love, and other positive emotions.** Some people find meaning in life through the pursuit of happiness and well-being.
> 
> * **Growing and learning throughout one's life.** Many people find meaning in life through the process of personal growth and development.
> 
> * **Connecting with something greater than oneself.** Some people find meaning in life through their faith, spirituality, or connection to nature.
> 
> Ultimately, the meaning of life is a personal and subjective matter. There is no right or wrong answer, and what gives life meaning for one person may not be the same for another. It is a question that many people spend their entire lives pondering and exploring.

## Single API Call

In [15]:
test_set_sample = test_set.sample(20)

test_set_sample['pred_label'] = ''

test_set_sample

,review,label,pred_label
798,It worked exactly as expected and the speaker ...,1,
342,"Easy setup using the app, which was also easy ...",1,
2686,"A great product for the price, works well!!",1,
1773,"Great speaker, great microphone response. Lov...",1,
1986,"We are a long time Amazon Prime family, many y...",1,
2146,So awesome to bring my older tv to life,1,
1440,Love the product idea. Super easy to set up & ...,1,
1572,"So far, I like it. It is better than I expect...",1,
427,Refurbished but just as good as the new ones t...,1,
561,Bought this for my son and it didn’t work. He ...,0,


In [19]:
# Convert the DataFrame to JSON using the to_json() method
# Changed 'clean_reviews' to 'review' as it is the column name in test_set_sample
json_data = test_set_sample[['review','pred_label']].to_json(orient='records')

# Print the JSON data
print(json_data)

[{"review":"It worked exactly as expected and the speaker has great sound. It is perfect for my classroom!","pred_label":""},{"review":"Easy setup using the app, which was also easy to install on my phone.  The sound quality while for listening to music on Pandora is wonderful!!","pred_label":""},{"review":"A great product for the price, works well!!","pred_label":""},{"review":"Great speaker, great microphone response.  Loving my purchase and the beginning of my smart home.","pred_label":""},{"review":"We are a long time Amazon Prime family, many years, but new to the Alexa world and thought the purchase of Alexa Plus, with hub, and included Phillips Hue light bulb would be a great start. Alexa does a lot of good things for us like Alarm in the morning, verbalize books, playing music on command, even linked to my Sirrius\/xm account. Good daily stuff. Good sound, but not Bose. But, the ads say &#34;Hub included&#34; so don't need anything else to operate devices. Well, not so. I under

In [20]:
prompt = f"""
You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.

```
{json_data}
```
"""

print(prompt)


You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.

```
[{"review":"It worked exactly as expected and the speaker has great sound. It is perfect for my classroom!","pred_label":""},{"review":"Easy setup using the app, which was also easy to install on my phone.  The sound quality while for listening to music on Pandora is wonderful!!","pred_label":""},{"review":"A great product for the price, works well!!","pred_label":""},{"review":"Great speaker, great microphone response.  Loving my purchase and the beginning of my smart home.","pred_label":""},{"review":"We 

In [ ]:
response = model.generate_content(prompt)

print(response.text)

In [ ]:
import json

# Clean the data by stripping the backticks
json_data = response.text.strip("`")

# Load the cleaned data and convert to DataFrame
data = json.loads(json_data)
df_sample = pd.DataFrame(data)

df_sample

,clean_reviews,pred_label
0,its okay but keeps dropping wifi,0
1,excelente su funcionamiento llama la atención ...,1
2,love that can shut the microphone off at night...,1
3,this echo dot is horrible the volume on my pho...,0
4,easy to set up and connect with smart devices ...,1
5,received the echo as gift needed another bluet...,1
6,menu doesn work says it can connect to amazon ...,0
7,the sound quality is not that good especially ...,0
8,the echo doesn come with built in battery so i...,0
9,love this it user friendly and better than any...,1


In [ ]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

test_set_sample['pred_label'] = df_sample['pred_label'].values
test_set_sample

,review,label,clean_reviews,pred_label
2874,its okay but keeps dropping wifi!,1,its okay but keeps dropping wifi,0
1212,Excelente su funcionamiento y llama la atenció...,1,excelente su funcionamiento llama la atención ...,1
2998,I love that I can shut the microphone off at n...,1,love that can shut the microphone off at night...,1
661,This Echo Dot is horrible. The volume on my ph...,0,this echo dot is horrible the volume on my pho...,0
2758,Easy to set up and connect with smart devices....,1,easy to set up and connect with smart devices ...,1
5,I received the echo as a gift. I needed anothe...,1,received the echo as gift needed another bluet...,1
2413,Menu doesn’t work. Says it can’t connect to am...,0,menu doesn work says it can connect to amazon ...,0
2045,"The sound quality is not that good, especially...",0,the sound quality is not that good especially ...,0
327,The echo doesn’t come with a built in battery ...,1,the echo doesn come with built in battery so i...,0
2273,I love this! It’s user friendly and better tha...,1,love this it user friendly and better than any...,1


In [ ]:
# Plotting confusion matrix on the predictions

from sklearn.metrics import confusion_matrix

y_true = test_set_sample["label"]
y_pred = test_set_sample["pred_label"]

confusion_matrix(y_true, y_pred)

array([[ 6,  0],
       [ 3, 11]])

### OpenAI API Config

In [ ]:
!pip install openai==0.27.0

In [ ]:
import openai
from google.colab import userdata

Openai_key=userdata.get('Openai_key')
openai.api_key  = Openai_key

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo-1106"):

  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(model=model,messages=messages,temperature=0)

  return response.choices[0].message["content"]

In [ ]:
prompt = "Why is the sky blue?"

chatgpt_response = get_completion(prompt)

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
chatgpt_response

### Batching API Calls(single shot)

In [ ]:
test_set.shape

(488, 3)

In [ ]:
test_set_total = test_set.sample(100)

test_set_total['pred_label'] = ''

test_set_total

,review,label,clean_reviews,pred_label
1002,"Very impressed with look, clarity of sound and...",1,very impressed with look clarity of sound and ...,
741,"It's like Siri, in fact, Siri answers more acc...",0,it like siri in fact siri answers more accurat...,
294,I use this product for enjoyment and I just lo...,1,use this product for enjoyment and just love it,
470,This item did not work. Certified refurbished ...,0,this item did not work certified refurbished s...,
3024,I was really happy with my original echo so i ...,0,was really happy with my original echo so thou...,
...,...,...,...,...
396,I bought an echo dot that had been refurbished...,0,bought an echo dot that had been refurbished b...,
1625,Our family bought several of these to help com...,0,our family bought several of these to help com...,
2754,Still learning everything she can do. Good fo...,1,still learning everything she can do good for ...,
521,Two weeks after setting it up it no longer wor...,0,two weeks after setting it up it no longer wor...,


In [ ]:
batches = []
batch_size = 50

for i in range(0, len(test_set_total), batch_size):
  batches.append(test_set_total[i : i + batch_size])  # Append batches instead of assigning

In [ ]:
import time

def gemini_completion_function(batch,current_batch,total_batch):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling Gemini API
  """

  print(f"Now processing batch#: {current_batch+1} of {total_batch}")

  json_data = batch[['clean_reviews','pred_label']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

  ```
  {json_data}
  ```
  """

  print(prompt)
  response = model.generate_content(prompt)
  time.sleep(5)

  return response

In [ ]:
batch_count = len(batches)
responses = []

for i in range(0,len(batches)):
  responses.append(gemini_completion_function(batches[i],i,batch_count))

Now processing batch#: 1 of 2
You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

  ```
  [{"clean_reviews":"invasive and scared the crap out of me for speaking when it was not spoken to creapy af","pred_label":""},{"clean_reviews":"not substitute for the original the echo dot performs well but sound quality is poor to listen it has to be at the highest level 10","pred_label":""},{"clean_reviews":"the echo dot disco

In [ ]:
import json

df_total = pd.DataFrame()  # Initialize an empty DataFrame

for response in responses:
  # Access the content of the response using .text
  json_data = response.text.strip("`")

  # Load the cleaned data and convert to DataFrame
  data = json.loads(json_data)
  df_temp = pd.DataFrame(data)

  # Append the DataFrame to the final DataFrame
  df_total = df_total.append(df_temp, ignore_index=True)

print(df_total)  # Display the final DataFrame

JSONDecodeError: Expecting value: line 2 column 9233 (char 9233)

In [ ]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

if len(test_set_total) == len(df_total):
    test_set_total['pred_label'] = df_total['pred_label'].values
    print(test_set_total)
else:
    print("Error: Length of 'df_total' does not match 'test_set_total'")

Error: Length of 'df_total' does not match 'test_set_total'


In [ ]:
# Plotting confusion matrix on the predictions

from sklearn.metrics import confusion_matrix

y_true = test_set_total["label"]
y_pred = test_set_total["pred_label"]

confusion_matrix(y_true, y_pred)

array([[44,  1],
       [ 5, 50]])

## Batching API Calls: ChatGPT (Few Shot)

In [ ]:
test_set.shape

(488, 3)

In [ ]:
test_set_total = test_set.sample(100)

test_set_total['pred_label'] = ''

test_set_total

,review,label,clean_reviews,pred_label
1386,Invasive and scared the crap out of me for spe...,0,invasive and scared the crap out of me for spe...,
556,Not substitute for the original. The echo dot...,1,not substitute for the original the echo dot p...,
544,The echo dot disconnects from the speaker abou...,0,the echo dot disconnects from the speaker abou...,
2852,Luv it. It's a great product.,1,luv it it great product,
1227,product turns on randomly and sometimes at night,0,product turns on randomly and sometimes at night,
...,...,...,...,...
2038,Pros- plenty loud for a large room with high c...,1,pros plenty loud for large room with high ceil...,
2806,,1,,
2628,,0,,
836,Not much features.,0,not much features,


In [ ]:
batches = []
batch_size = 50

for i in range(0, len(test_set_total), batch_size):
  batches.append(test_set_total[i : i + batch_size])  # Append batches instead of assigning

In [ ]:
import time

def gpt_completion_function(batch,current_batch,total_batch,train_sample,model="gpt-3.5-turbo-1106"):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling GPT API
  """

  print(f"Now processing batch#: {current_batch+1} of {total_batch}")

  json_data = batch[['clean_reviews','pred_label']].to_json(orient='records')

  sample_json_data = train_sample[['clean_reviews','label']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).
  Examples of good Sentiment Analysis Classification are provided between separator ####.
  These examples are for your reference, not to be included in your final output.

  ```
  {json_data}
  ```
  ####
  {sample_json_data}
  ####
  """

  print(prompt)

  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(model=model,messages=messages,temperature=0)
  time.sleep(5)
  return response.choices[0].message["content"]

In [ ]:
train_sample = train_set.sample(4)

batch_count = len(batches)
responses = []

for i in range(0,len(batches)):
  responses.append(gpt_completion_function(batches[i],i,batch_count,train_sample))

Now processing batch#: 1 of 2
You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).
  Examples of good Sentiment Analysis Classification are provided between separator ####.
  These examples are for your reference, not to be included in your final output.

  ```
  [{"clean_reviews":"invasive and scared the crap out of me for speaking when it was not spoken to creapy af","pred_label":""},{"clean_reviews":"not substitute

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
import json

df_total = pd.DataFrame()  # Initialize an empty DataFrame

for response in responses:
  # Clean the data by stripping the backticks
  json_data = response.strip("`")

  # Load the cleaned data and convert to DataFrame
  data = json.loads(json_data)
  df_temp = pd.DataFrame(data)

  # Append the DataFrame to the final DataFrame
  df_total = df_total.append(df_temp, ignore_index=True)

print(df_total)  # Display the final DataFrame

Empty DataFrame
Columns: []
Index: []


In [ ]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

test_set_total['pred_label'] = df_total['pred_label'].values
test_set_total

KeyError: 'pred_label'

In [ ]:
# Plotting confusion matrix on the predictions

from sklearn.metrics import confusion_matrix, accuracy_score

y_true = test_set_total["label"]
y_pred = test_set_total["pred_label"]

print(confusion_matrix(y_true, y_pred))
print(f"\nAccuracy: {accuracy_score(y_true, y_pred)}")

TypeError: Labels in y_true and y_pred should be of the same type. Got y_true=[0 1] and y_pred=['']. Make sure that the predictions provided by the classifier coincides with the true labels.